In [1]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

import time

import warnings
warnings.filterwarnings('ignore')

# Load Data


In [2]:
# load series ID
df_id = pd.read_csv('ids.csv')
df_id

,id,title,freq,start,end,sa,category
0,RPI,Real Personal Income,Monthly,1/1/1959,11/1/2023,Seasonally Adjusted Annual Rate,Income
1,W875RX1,Real personal income excluding current transfe...,Monthly,1/1/1959,11/1/2023,Seasonally Adjusted Annual Rate,Income
2,DSPIC96,Real Disposable Personal Income,Monthly,1/1/1959,11/1/2023,Seasonally Adjusted,Income
3,PIROA,Personal Income Receipts on Assets,Monthly,1/1/1959,11/1/2023,Seasonally Adjusted,Income
4,PSAVERT,Personal Saving Rate,Monthly,1/1/1959,11/1/2023,Seasonally Adjusted Annual Rate,Income
...,...,...,...,...,...,...,...
164,CCUSSP01JPM650N,Currency Conversions: US Dollar Exchange Rate:...,Monthly,1/1/1960,11/1/2023,Not Seasonally Adjusted,Forex
165,CCUSSP01DEM650N,Currency Conversions: US Dollar Exchange Rate:...,Monthly,1/1/1960,11/1/2023,Not Seasonally Adjusted,Forex
166,CCUSSP01CAM650N,Currency Conversions: US Dollar Exchange Rate:...,Monthly,1/1/1960,11/1/2023,Not Seasonally Adjusted,Forex
167,CCUSSP01MXM650N,Currency Conversions: US Dollar Exchange Rate:...,Monthly,1/1/1960,11/1/2023,Not Seasonally Adjusted,Forex


# FRED API

Request API key:
https://fredaccount.stlouisfed.org/login/secure/

API documentation:
https://fred.stlouisfed.org/docs/api/fred/  

Tutorial: https://mortada.net/python-api-for-fred.html

The FRED API has a limit of 1000 items per request, with a rate limit of 120 requests per minute. If you receive a timeout error, please wait a while before downloading.

In [5]:
#%%capture
#!pip install fredapi

In [3]:
# Input API key
from fredapi import Fred as fred
api_key='1c9b36f821931d47af2a923286a83447'
fred = fred(api_key=api_key)

In [4]:
def get_meta_data(search='United States', limit = 1000, order_by = 'popularity', freq='Monthly', cutoff='1/1/2023', sa=True):
  '''
  Returns a metadata (dataframe) for economic indicators from FRED API.
  ----------
  Parameters
  ----------
  country: country name (str) or search keyword (e.g.,'GDP')
  limit: default 1000
  order_by: search in specified order:'seach_rank', 'series_id', 'title', 'units', 'frequency','seasonal_adjustment',
  realtime_start', 'realtime_end', 'last_updated', 'observation_start', 'observation_end','popularity'
  freq: default 'Monthly'. Valid options are 'Daily', 'Monthly', 'Quarterly', 'Annual'
  cutoff: date ('MM/DD/YYYY'). Remove discontinued series
  sa: returns all if False. Options are 'Not Seasonally Adjusted', 'Seasonally Adjusted', 'Seasonally Adjusted Annual Rate', 'Smoothed Seasonally Adjusted'
  '''
  data = fred.search(search)
  data = data[(data['frequency']==freq) & (data['observation_end']>cutoff)]
  if sa == True:
    try:
      data = data[data['seasonal_adjustment'] !='Not Seasonally Adjusted']
    except:
      time.sleep(60)
  return data

get_meta_data()


,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
CPIAUCSL,CPIAUCSL,2024-01-21,2024-01-21,Consumer Price Index for All Urban Consumers: ...,1947-01-01,2023-12-01,Monthly,M,Index 1982-1984=100,Index 1982-1984=100,Seasonally Adjusted,SA,2024-01-11 07:40:01-06:00,94,The Consumer Price Index for All Urban Consume...
M2SL,M2SL,2024-01-21,2024-01-21,M2,1959-01-01,2023-11-01,Monthly,M,Billions of Dollars,Bil. of $,Seasonally Adjusted,SA,2023-12-26 12:03:01-06:00,92,"Before May 2020, M2 consists of M1 plus (1) sa..."
UNRATE,UNRATE,2024-01-21,2024-01-21,Unemployment Rate,1948-01-01,2023-12-01,Monthly,M,Percent,%,Seasonally Adjusted,SA,2024-01-12 10:28:02-06:00,93,The unemployment rate represents the number of...
CSUSHPISA,CSUSHPISA,2024-01-21,2024-01-21,S&P CoreLogic Case-Shiller U.S. National Home ...,1987-01-01,2023-10-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Seasonally Adjusted,SA,2023-12-26 08:12:02-06:00,77,"For more information regarding the index, plea..."
M1SL,M1SL,2024-01-21,2024-01-21,M1,1959-01-01,2023-11-01,Monthly,M,Billions of Dollars,Bil. of $,Seasonally Adjusted,SA,2023-12-26 12:03:04-06:00,82,"Before May 2020, M1 consists of (1) currency o..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CUSR0000SEHC,CUSR0000SEHC,2024-01-21,2024-01-21,Consumer Price Index for All Urban Consumers: ...,1983-01-01,2023-12-01,Monthly,M,Index Dec 1982=100,Index Dec 1982=100,Seasonally Adjusted,SA,2024-01-11 07:39:04-06:00,55,None
SDXRSA,SDXRSA,2024-01-21,2024-01-21,S&P CoreLogic Case-Shiller CA-San Diego Home P...,1987-01-01,2023-10-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Seasonally Adjusted,SA,2023-12-26 08:11:06-06:00,55,"For more information regarding the index, plea..."
LNS14000024,LNS14000024,2024-01-21,2024-01-21,Unemployment Rate - 20 Yrs. & over,1948-01-01,2023-12-01,Monthly,M,Percent,%,Seasonally Adjusted,SA,2024-01-05 07:45:32-06:00,55,The series comes from the 'Current Population ...


In [8]:
def get_series_data(id, observation_start='1/1/1959'):
  '''
  Returns a dataframe with time series data retrieved from the FRED database
  ----------
  Parameters
  ----------
  id : FRED series id (list or array)
  observation_start = 'MM/DD/YYYY'
  '''
  dataset = {}
  for i in id:
    try:
      dataset[i] = fred.get_series(i, observation_start=observation_start)
    except:
      time.sleep(60)
  return pd.DataFrame(dataset)

# Recession Labels

In [9]:
# NBER Recession Indicator
fred.get_series_info('USREC')

id                                                                       USREC
realtime_start                                                      2024-01-06
realtime_end                                                        2024-01-06
title                        NBER based Recession Indicators for the United...
observation_start                                                   1854-12-01
observation_end                                                     2023-12-01
frequency                                                              Monthly
frequency_short                                                              M
units                                                                  +1 or 0
units_short                                                            +1 or 0
seasonal_adjustment                                    Not Seasonally Adjusted
seasonal_adjustment_short                                                  NSA
last_updated                                        

In [10]:
# Regime indicator: Recession = 1, Normal = 0
recession = fred.get_series('USREC')
recession.to_csv('recession.csv', index_label='Date', header=['Regime'])

# Create Dataset

In [11]:
# Download economic indicator time-series data

start_time = time.time()

df = get_series_data(df_id.id, observation_start='1/1/1959')

print(f'Download completed in {round((time.time() - start_time) / 60, 2)} minutes.')
print(f'nubmer of indicators: ', df.shape[1])
print(f'number of months: ', df.shape[0])

Download completed in 2.34 minutes.
nubmer of indicators:  169
number of months:  780


In [ ]:
df.to_csv('dataset.csv')
df.head()

,RPI,W875RX1,DSPIC96,PIROA,PSAVERT,DPCERA3M086SBEA,UMCSENT,PCE,W055RC1,RETAIL,...,PPIIDC,WTISPLC,IPMINE,M1109BUSM293NNBR,CCUSSP01GBM650N,CCUSSP01JPM650N,CCUSSP01DEM650N,CCUSSP01CAM650N,CCUSSP01MXM650N,CCUSSP02CNM650N
1959-01-01,2583.560,2426.0,2318.4,38.4,11.3,15.188,NaN,306.1,40.2,17583.0,...,30.3,3.00,62.6418,592.29,NaN,NaN,NaN,NaN,NaN,NaN
1959-02-01,2593.596,2434.8,2325.4,38.9,10.6,15.346,NaN,309.6,40.8,17712.0,...,30.4,3.00,63.0540,590.72,NaN,NaN,NaN,NaN,NaN,NaN
1959-03-01,2610.396,2452.7,2338.7,39.2,10.3,15.491,NaN,312.7,41.3,17860.0,...,30.5,2.97,63.1913,609.12,NaN,NaN,NaN,NaN,NaN,NaN
1959-04-01,2627.446,2470.0,2353.8,39.7,11.2,15.435,NaN,312.2,41.7,17871.0,...,30.5,2.97,65.0459,616.99,NaN,NaN,NaN,NaN,NaN,NaN
1959-05-01,2642.720,2486.4,2366.6,40.1,10.6,15.622,95.3,316.1,42.1,18011.0,...,30.5,2.97,66.4883,630.80,NaN,NaN,NaN,NaN,NaN,NaN
